## Players-rating analysis

In [13]:
import pandas as pd

import MySQLdb

db = MySQLdb.connect(
    "db.fastcamp.us",  
    "root",  
    "dkstncks", 
    "football", 
    charset='utf8'
    )

## 1) FW

In [34]:
SQL_QUERY = """
    SELECT goals, assists, yel, red, spg, ps_x, motm, aw, tackles, inter, fouls, offsides, clear, drb,
           blocks, owng, keyp, fouled, off, disp, unstch, avgp, rating
    FROM player
    WHERE
        position LIKE '%FW%'
        AND mins > '400'   # 400분 미만인 애들 빼기
    ;
"""

fw_df1 = pd.read_sql(SQL_QUERY, db)

In [35]:
fw_df1.tail()

,goals,assists,yel,red,spg,ps_x,motm,aw,tackles,inter,...,drb,blocks,owng,keyp,fouled,off,disp,unstch,avgp,rating
510,5,7,5,0,1.2,84.0,2,0.6,2.8,0.6,...,2.0,0.2,0,1.8,1.1,0.1,0.8,0.7,1.8,7.14
511,12,6,8,0,2.4,77.8,2,1.5,0.8,0.2,...,0.5,0.1,0,1.1,1.8,1.2,2.5,2.9,1.1,7.08
512,1,0,0,0,0.9,86.2,1,0.4,1.0,0.9,...,0.7,0.0,0,0.4,0.6,0.1,0.9,1.2,0.4,6.65
513,8,3,9,0,2.4,64.1,0,1.9,0.7,0.3,...,0.2,0.1,0,0.4,1.8,1.1,1.8,3.1,0.4,6.67
514,2,1,0,0,1.4,82.2,2,1.4,0.3,0.3,...,0.3,0.0,0,0.7,1.4,0.4,1.6,2.1,0.7,6.54


In [25]:
import statsmodels.api as sm
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import StandardScaler

In [41]:
scaler = StandardScaler(with_mean=False)
X_scaled = StandardScaler(fw_df1.ix[:,:-1])

In [42]:
fw_df1 = sm.add_constant(fw_df1)

In [43]:
model1 = sm.OLS(fw_df1.ix[:,-1], fw_df1.ix[:,:-1])
result1 = model1.fit()
print(result1.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.905
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     224.7
Date:                Thu, 23 Jun 2016   Prob (F-statistic):          7.57e-237
Time:                        12:24:37   Log-Likelihood:                 358.41
No. Observations:                 515   AIC:                            -672.8
Df Residuals:                     493   BIC:                            -579.5
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          5.6150      0.079     71.011      0.0

In [50]:
# offsides, blocks, fouls, disp , clear 뺌
# clear 같은 경우는 공격수 대부분이 0인 값을 가지고 있어서 뺌

In [47]:
SQL_QUERY = """
    SELECT goals, assists, yel, red, spg, ps_x, motm, aw, tackles, inter, drb,
           owng, keyp, fouled, off, unstch, avgp, rating
    FROM player
    WHERE
        position LIKE '%FW%'
        AND mins > '400'   # 400분 미만인 애들 빼기
    ;
"""

fw_df2 = pd.read_sql(SQL_QUERY, db)

In [48]:
scaler = StandardScaler(with_mean=False)
X_scaled = StandardScaler(fw_df2.ix[:,:-1])

fw_df2 = sm.add_constant(fw_df2)

model2 = sm.OLS(fw_df2.ix[:,-1], fw_df2.ix[:,:-1])
result2 = model2.fit()
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.905
Model:                            OLS   Adj. R-squared:                  0.902
Method:                 Least Squares   F-statistic:                     295.5
Date:                Thu, 23 Jun 2016   Prob (F-statistic):          4.04e-242
Time:                        12:27:02   Log-Likelihood:                 356.46
No. Observations:                 515   AIC:                            -678.9
Df Residuals:                     498   BIC:                            -606.8
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          5.6156      0.078     72.252      0.0

* 생각보다 tackles 와 interceptions 가 rating 에 영향을 많이 끼치는 것으로 나타남.

## 2) MF

In [51]:
SQL_QUERY = """
    SELECT goals, assists, yel, red, spg, ps_x, motm, aw, tackles, inter, fouls, offsides, clear, drb,
           blocks, owng, keyp, fouled, off, disp, unstch, crosses, longb, thrb, rating
    FROM player
    WHERE
        position LIKE '%M(%'
        OR position LIKE '%AM%'
        OR position LIKE 'DMC'
        AND mins > '400'
    ;
"""

mf_df1 = pd.read_sql(SQL_QUERY, db)

In [54]:
# 미드필더의 경우 패스의 수가 다른 포지션 보다 많기 때문에 crosses, longb, thrb 를 포함.

In [52]:
mf_df1.tail()

,goals,assists,yel,red,spg,ps_x,motm,aw,tackles,inter,...,owng,keyp,fouled,off,disp,unstch,crosses,longb,thrb,rating
1460,0,2,2,1,0.7,77.6,0,0.7,1.3,1.0,...,0,0.7,1.0,0.1,1.9,1.5,77.6,0.3,0.6,6.54
1461,0,1,0,0,0.9,86.6,0,0.0,0.7,1.1,...,0,0.3,1.3,0.3,1.0,0.9,86.6,0.7,0.0,6.37
1462,0,0,2,0,0.5,85.8,0,0.5,1.1,1.1,...,0,0.2,0.5,0.0,0.5,1.1,85.8,0.1,1.3,6.29
1463,0,0,0,0,0.0,83.7,0,1.0,1.0,0.3,...,0,0.6,0.0,0.0,0.7,0.4,83.7,0.1,1.7,6.26
1464,0,0,0,0,0.0,93.3,0,0.0,0.0,0.0,...,0,0.0,2.0,0.0,0.0,1.0,93.3,0.0,0.0,6.01


In [53]:
scaler = StandardScaler(with_mean=False)
X_scaled = StandardScaler(mf_df1.ix[:,:-1])

mf_df1 = sm.add_constant(mf_df1)

model1 = sm.OLS(mf_df1.ix[:,-1], mf_df1.ix[:,:-1])
result1 = model1.fit()
print(result1.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.863
Model:                            OLS   Adj. R-squared:                  0.861
Method:                 Least Squares   F-statistic:                     396.2
Date:                Thu, 23 Jun 2016   Prob (F-statistic):               0.00
Time:                        12:38:45   Log-Likelihood:                 865.12
No. Observations:                1465   AIC:                            -1682.
Df Residuals:                    1441   BIC:                            -1555.
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          5.6534      0.038    149.955      0.0

In [55]:
# blocks 과 off ( 오프사이드를 일으킨 횟수 ) 상관관계가 낮아 제거함.

In [56]:
SQL_QUERY = """
    SELECT goals, assists, yel, red, spg, ps_x, motm, aw, tackles, inter, fouls, offsides, clear, drb,
           owng, keyp, fouled, disp, unstch, crosses, longb, thrb, rating
    FROM player
    WHERE
        position LIKE '%M(%'
        OR position LIKE '%AM%'
        OR position LIKE 'DMC'
        AND mins > '400'
    ;
"""

mf_df2 = pd.read_sql(SQL_QUERY, db)

In [57]:
scaler = StandardScaler(with_mean=False)
X_scaled = StandardScaler(mf_df2.ix[:,:-1])

mf_df2 = sm.add_constant(mf_df2)

model2 = sm.OLS(mf_df2.ix[:,-1], mf_df2.ix[:,:-1])
result2 = model2.fit()
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.863
Model:                            OLS   Adj. R-squared:                  0.861
Method:                 Least Squares   F-statistic:                     434.5
Date:                Thu, 23 Jun 2016   Prob (F-statistic):               0.00
Time:                        12:45:44   Log-Likelihood:                 865.12
No. Observations:                1465   AIC:                            -1686.
Df Residuals:                    1443   BIC:                            -1570.
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          5.6535      0.037    150.935      0.0

## 3) DF

In [62]:
SQL_QUERY = """
    SELECT goals, assists, yel, red, spg, ps_x, motm, aw, tackles, inter, fouls, offsides, clear, drb,
           blocks, owng, keyp, fouled, off, disp, unstch, avgp, crosses, longb, thrb, rating
    FROM player
    WHERE
        position LIKE '%D(%'
        OR position LIKE '%DF%'
        AND mins > '400'
    ;
"""

df_df1 = pd.read_sql(SQL_QUERY, db)

In [59]:
scaler = StandardScaler(with_mean=False)
X_scaled = StandardScaler(df_df1.ix[:,:-1])

df_df1 = sm.add_constant(df_df1)

model1 = sm.OLS(df_df1.ix[:,-1], df_df1.ix[:,:-1])
result1 = model1.fit()
print(result1.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     170.9
Date:                Thu, 23 Jun 2016   Prob (F-statistic):               0.00
Time:                        13:07:00   Log-Likelihood:                 632.81
No. Observations:                1209   AIC:                            -1218.
Df Residuals:                    1185   BIC:                            -1095.
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          5.3503      0.049    108.551      0.0

In [61]:
SQL_QUERY = """
    SELECT goals, assists, red, spg, ps_x, motm, aw, tackles, inter, fouls, offsides, clear, drb,
           blocks, owng, keyp, fouled, disp, avgp, crosses, longb, thrb, rating
    FROM player
    WHERE
        position LIKE '%D(%'
        OR position LIKE '%DF%'
        AND mins > '400'
    ;
"""

df_df2 = pd.read_sql(SQL_QUERY, db)

In [63]:
scaler = StandardScaler(with_mean=False)
X_scaled = StandardScaler(df_df2.ix[:,:-1])

df_df2 = sm.add_constant(df_df2)

model2 = sm.OLS(df_df2.ix[:,-1], df_df2.ix[:,:-1])
result2 = model2.fit()
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     197.0
Date:                Thu, 23 Jun 2016   Prob (F-statistic):               0.00
Time:                        13:08:45   Log-Likelihood:                 632.77
No. Observations:                1209   AIC:                            -1224.
Df Residuals:                    1188   BIC:                            -1116.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          5.3510      0.049    109.195      0.0

In [66]:
SQL_QUERY = """
    SELECT goals, assists, red, spg, ps_x, motm, aw, tackles, inter, fouls, offsides, clear, drb,
           blocks, owng, keyp, fouled, disp, crosses, longb, thrb, rating
    FROM player
    WHERE
        position LIKE '%D(%'
        OR position LIKE '%DF%'
        AND mins > '400'
    ;
"""

df_df3 = pd.read_sql(SQL_QUERY, db)

In [67]:
scaler = StandardScaler(with_mean=False)
X_scaled = StandardScaler(df_df3.ix[:,:-1])

df_df3 = sm.add_constant(df_df3)

model3 = sm.OLS(df_df3.ix[:,-1], df_df3.ix[:,:-1])
result3 = model3.fit()
print(result3.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     197.0
Date:                Thu, 23 Jun 2016   Prob (F-statistic):               0.00
Time:                        13:10:40   Log-Likelihood:                 632.77
No. Observations:                1209   AIC:                            -1224.
Df Residuals:                    1188   BIC:                            -1116.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          5.3510      0.049    109.195      0.0